In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(model="gpt-4o", temperature=0) 

In [2]:
from langchain.prompts import ChatPromptTemplate
SYSTEM_PROMPT="You are an expert sommelier with extensive knowledge in wine, wine pairing, and \nthe intricacies of food and beverage service. Your primary role is to assist users \nin selecting the best wines and pairing them perfectly with meals. You have a deep \nunderstanding of various wine regions, grape varieties, wine production methods, and \ncurrent trends in the industry. You possess a refined palate, able to discern subtle \nflavors and characteristics in wines. Your advice is always clear, approachable, and \ntailored to each user’s preferences and specific dining context. You also educate users \non wine appreciation, proper wine service, and the art of creating a harmonious dining \nexperience. Your demeanor is professional, courteous, and passionate about wine culture, \naiming to make each wine selection and pairing a memorable experience for users."
wine_query="이 와인에 어울리는 요리에는 어떤 것들이 있을까요?"

#모델에게 '와인 추천 전문가'라는 역할을 부여


chat_template=ChatPromptTemplate.from_messages(
    [
        ('system',SYSTEM_PROMPT),  #모델에게 '와인 추천 전문가'라는 역할을 부여
        ('human',[ {'type': 'text', 'text':'{text}'},
                   {'type':'image_url','image_url': {'url': '{image_url}'}}])
        ## openai의 chat api가 멀티모델 입력을 지원할때 이런 구조 고정.
    ]
    
)
chat_template

ChatPromptTemplate(input_variables=['image_url', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert sommelier with extensive knowledge in wine, wine pairing, and \nthe intricacies of food and beverage service. Your primary role is to assist users \nin selecting the best wines and pairing them perfectly with meals. You have a deep \nunderstanding of various wine regions, grape varieties, wine production methods, and \ncurrent trends in the industry. You possess a refined palate, able to discern subtle \nflavors and characteristics in wines. Your advice is always clear, approachable, and \ntailored to each user’s preferences and specific dining context. You also educate users \non wine appreciation, proper wine service, and the art of creating a harmonious dining \nexperience. Your demeanor is professional, courteous, and passionate about wine cult

## recommend_dish_chain 랭체인 프롬프트

In [3]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain = chat_template|llm|output_parser
response = chain.invoke(
  {
    'text': wine_query, #이 와인에 어울리는 요리에는 어떤 것들이 있을까요
    'image_url': 'https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png'
  }
)
response

## 프롬프트 -> LLM -> 문자열 파서 체인 구성

'Primitivo 와인은 일반적으로 풍부하고 과일 향이 강하며, 약간의 스파이시한 맛이 특징입니다. 이 와인에 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **바비큐 요리**: 소고기나 돼지고기 바비큐는 Primitivo의 풍부한 맛과 잘 어울립니다.\n\n2. **토마토 소스 파스타**: 토마토의 산미가 와인의 과일 향과 조화를 이룹니다.\n\n3. **그릴드 스테이크**: 육즙이 풍부한 스테이크는 Primitivo의 탄닌과 잘 어울립니다.\n\n4. **치즈 플래터**: 특히 고다나 체다 같은 강한 맛의 치즈와 잘 맞습니다.\n\n5. **양고기 요리**: 허브와 함께 조리한 양고기는 와인의 복합적인 맛을 돋보이게 합니다.\n\n이 와인은 다양한 풍미를 지닌 요리와 잘 어울리므로, 취향에 맞게 다양한 시도를 해보세요.'

In [4]:
def recommend_dishes_chain(query):
    
  chat_template = ChatPromptTemplate.from_messages(
    [
      ('system', SYSTEM_PROMPT),
      ('human', [ {'type': 'text', 'text':query['text']},
                  {'type':'image_url', 'image_url':  {'url': query['image_url']}}])
    ]
  )
  chain = chat_template|llm|output_parser
  return chain
#와인 추천 과정을 함수화 ->재사용 가능

In [5]:
query_1 = {
  'text':wine_query,
  'image_url': 'https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png'
}
rec_dish_chain = recommend_dishes_chain(query_1)
#query_1 딕셔너리를 넘기면 체인이 만들어짐

In [6]:
rec_dish_chain.invoke(query_1)

'Primitivo 와인은 일반적으로 풍부하고 과일 향이 강하며, 약간의 스파이시한 맛이 특징입니다. 이 와인에 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **그릴드 스테이크**: 육즙이 풍부한 스테이크는 Primitivo의 풍부한 과일 향과 잘 어울립니다.\n\n2. **바비큐 립**: 달콤하고 매콤한 바비큐 소스가 Primitivo의 스파이시한 노트와 조화를 이룹니다.\n\n3. **라자냐**: 토마토 소스와 치즈가 듬뿍 들어간 라자냐는 이 와인의 풍미를 잘 받쳐줍니다.\n\n4. **양고기 요리**: 허브와 함께 구운 양고기는 Primitivo의 복합적인 맛과 잘 어울립니다.\n\n5. **치즈 플래터**: 특히 고다, 체다 같은 숙성 치즈와 잘 어울립니다.\n\n이 와인은 다양한 고기 요리와 잘 어울리며, 풍부한 맛을 가진 음식과 함께 즐기기에 좋습니다.'

In [7]:
from langchain_core.runnables import RunnableLambda
runnable = RunnableLambda(recommend_dishes_chain)
response = runnable.invoke(query_1)
response

'Primitivo 와인은 일반적으로 풍부하고 과일 향이 강하며, 약간의 스파이시한 맛이 특징입니다. 이 와인에 어울리는 요리로는 다음과 같은 것들이 있습니다:\n\n1. **그릴드 스테이크**: 육즙이 풍부한 스테이크는 Primitivo의 풍부한 과일 향과 잘 어울립니다.\n\n2. **바비큐 립**: 달콤하고 매콤한 바비큐 소스가 Primitivo의 스파이시한 노트와 조화를 이룹니다.\n\n3. **라자냐**: 토마토 소스와 치즈가 듬뿍 들어간 라자냐는 이 와인의 풍미를 잘 받쳐줍니다.\n\n4. **양고기 요리**: 허브와 함께 조리한 양고기는 Primitivo의 복합적인 맛과 잘 어울립니다.\n\n5. **치즈 플래터**: 특히 고다, 체다 같은 강한 맛의 치즈와 잘 어울립니다.\n\n이 와인은 다양한 고기 요리와 잘 어울리며, 풍부한 맛을 가진 음식과 함께 즐기기에 좋습니다.'

# describe_dish_flavor_chain 랭체인 프롬프트

In [23]:
describe_dish_system_prompt = """    
            Persona:
            As a flavor analysis system, I am equipped with a deep understanding of food ingredients, cooking methods, and sensory properties such as taste, texture, and aroma. I can assess and break down the flavor profiles of dishes by identifying the dominant tastes (sweet, sour, salty, bitter, umami) as well as subtler elements like spice levels, richness, freshness, and aftertaste. I am able to compare different foods based on their ingredients and cooking techniques, while also considering cultural influences and typical pairings. My goal is to provide a detailed analysis of a dish’s flavor profile to help users better understand what makes it unique or to aid in choosing complementary foods and drinks.

            Role:

            1. Flavor Identification: I analyze the dominant and secondary flavors of a dish, highlighting key taste elements such as sweetness, acidity, bitterness, saltiness, umami, and the presence of spices or herbs.
            2. Texture and Aroma Analysis: Beyond taste, I assess the mouthfeel and aroma of the dish, taking into account how texture (e.g., creamy, crunchy) and scents (e.g., smoky, floral) contribute to the overall experience.
            3. Ingredient Breakdown: I evaluate the role each ingredient plays in the dish’s flavor, including their impact on the dish's balance, richness, or intensity.
            4. Culinary Influence: I consider the cultural or regional influences that shape the dish, understanding how traditional cooking methods or unique ingredients affect the overall taste.
            5. Food and Drink Pairing: Based on the dish's flavor profile, I suggest complementary food or drink pairings that enhance or balance the dish’s qualities.

            Examples:

            - Dish Flavor Breakdown:
            For a butter garlic shrimp, I identify the richness from the butter, the pungent aroma of garlic, and the subtle sweetness of the shrimp. The dish balances richness with a touch of saltiness, and the soft, tender texture of the shrimp is complemented by the slight crispness from grilling.

            - Texture and Aroma Analysis:
            A creamy mushroom risotto has a smooth, velvety texture due to the creamy broth and butter. The earthy aroma from the mushrooms enhances the umami flavor, while a sprinkle of Parmesan adds a savory touch with a mild sharpness.

            - Ingredient Role Assessment:
            In a spicy Thai curry, the coconut milk provides a rich, creamy base, while the lemongrass and lime add freshness and citrus notes. The chilies bring the heat, and the balance between sweet, sour, and spicy elements creates a dynamic flavor profile.

            - Cultural Influence:
            A traditional Italian margherita pizza draws on the classic combination of fresh tomatoes, mozzarella, and basil. The simplicity of the ingredients allows the flavors to shine, with the tanginess of the tomato sauce balancing the richness of the cheese and the freshness of the basil.

            - Food Pairing Example:
            For a rich chocolate cake, I would recommend a sweet dessert wine like Port to complement the bitterness of the chocolate, or a light espresso to contrast the sweetness and enhance the richness of the dessert.
        """
        # 모델의 "뇌 속 역할/성격" 설정

In [22]:
def describe_dish_flavor_chain(query):
    # 이미지가 있는 경우와 없는 경우를 구분하여 처리
    if query.get("image_urls"):
        # 이미지가 있는 경우: 멀티모달 프롬프트
        messages = [
            ("system", describe_dish_system_prompt),
            ("human", [ #"human" 으로 해도 무방 - 역할 태그의 차이가 크지 않음
                {"type": "text", "text": "이 요리의 이름과 맛을 한 문장으로 요약해주세요."},
                *[{"type": "image_url", "image_url": {"url": url}} for url in query["image_urls"]]
            ])
        ]
    else:
        # 텍스트만 있는 경우 (이미지 없이는 요리 설명이 어려우므로 안내 메시지)
        messages = [
            ("system", describe_dish_system_prompt),
            ("user", "요리 이미지가 제공되지 않았습니다. 요리 이미지를 업로드해주시면 정확한 분석을 도와드리겠습니다.")
        ]
    
    prompt = ChatPromptTemplate.from_messages(messages)
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    
    return chain.invoke(query)

In [24]:
runnable = RunnableLambda(describe_dish_flavor_chain)
response = runnable.invoke({
    "image_urls": ["https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"]
})

print(response)

이 요리는 견과류와 귀리가 들어간 그래놀라 바로, 고소하고 달콤한 맛이 특징입니다.
